In [ ]:
#####
library(emBayes)
#class Metrics:
    # tp: int; fp: int; fn: int; tn: int
    # tpr: float; fpr: float
    # selected: int; brier: float

In [ ]:
### Read multiple CSV data
read_multiple_csv_data <- function(data_dir, file_pattern = "dataX_", num_files = 1) {
  # Read multiple CSV files and combine them
  all_dataX <- list()
  all_dataY <- list()
  
  for (i in 0:(num_files-1)) {
    # Construct file paths
    dataX_file <- file.path(data_dir, paste0("dataX_", i, ".csv"))
    dataY_file <- file.path(data_dir, paste0("dataY_", i, ".csv"))
    
    # Read CSV files
    dataX <- read.csv(dataX_file)
    dataY <- read.csv(dataY_file)
    
    # Store in lists (R list indexing starts from 1, so use i+1)
    all_dataX[[i+1]] <- dataX
    all_dataY[[i+1]] <- dataY
  }
  
  # Return combined data
  return(list(
    dataX_list = all_dataX,
    dataY_list = all_dataY,
    num_files = num_files
  ))
}

# Example usage:
data_dir <- './R_test_bench/'
combined_data <- read_multiple_csv_data(data_dir, "dataX_", 30)


In [ ]:
embayes_fit <- function(y, X, quant_def=0.7){
    t0 <- seq(0.1,0.15,length.out=2)
    t1 <- seq(0.1,0.5,length.out=2)
    CV <- cv.emBayes(y,clin=NULL,X,W=NULL,nt=NULL,group=NULL,quant=quant_def,t0,t1,k=5,
                    func="ssLASSO",error=0.01,maxiter=1)
    s0 <- CV$CL.s0
    s1 <- CV$CL.s1
    EM <-emBayes(y,
                clin = NULL,
                X,
                W = NULL,
                nt = NULL,
                group = NULL,
                quant = quant_def,
                s0 = s0,
                s1 = s1,
                func = "ssLASSO",
                error = 0.01,
                maxiter = 1)
    return(EM)
}
# EM_result <- embayes_fit(y, X, quant)

### Run the embayes_fit function for all the data
n = combined_data$num_files
result_list <- list()
start_time <- Sys.time()
for (i in 1:n){
    X = as.matrix(combined_data$dataX_list[[i]])
    y = combined_data$dataY_list[[i]]$y
    y <- y[1:dim(X)[1]]
    EM_result <- embayes_fit(y, X)
    result_list[[i]] <- EM_result$beta
}
consumed_time <- Sys.time() - start_time
ave_time <- consumed_time/n
ave_time


Time difference of 0.2979199 mins

In [ ]:
### Write all result in CSV file
write.csv(result_list, file = "./R_test_bench/result_EM_T_100.csv")
# write.csv(result_list, file = "./R_test_bench/result_EM_T_50.csv")
# write.csv(result_list, file = "./R_test_bench/result_EM_Gamma_100.csv")
# write.csv(result_list, file = "./R_test_bench/result_EM_Gamma_50.csv")
# write.csv(result_list, file = "./R_test_bench/result_EM_Gau_100.csv")
# write.csv(result_list, file = "./R_test_bench/result_EM_Gau_50.csv")
